## Ejercicio del Parcial

Un rascacielos no tiene más de 100 pisos, numerados de 0 a 99. Tiene N ascensores que viajan hacia arriba y hacia abajo a velocidades (posiblemente) diferentes. El elevador toma entre 1 y 100 segundos para viajar entre dos pisos adyacentes (subiendo o bajando). Los elevadores no necesariamente se detienen en cada uno de los pisos. Lo que es peor, no todos los pisos son necesariamente accesibles por un elevador. Suponga que usted está en el piso 0 y le gustaría llegar al piso k lo más rápido posible. Suponga que no necesita esperar para abordar el primer elevador en el que se sube y (por simplicidad) la operación de cambiar de elevador en algún piso siempre toma exactamente un minuto. Por supuesto, ambos ascensores tienen que parar en ese piso. Tiene prohibido utilizar la escalera. Nadie más está en el ascensor con usted, por lo que no tiene que detenerse si no quiere hacerlo. Calcule la cantidad mínima de segundos requeridos para llegar del piso 0 al piso k (pasar el piso k mientras se encuentra dentro de un ascensor que no se detiene allí no cuenta como "llegar al piso k").

Consideraciones:
Modele la solución utilizando un grafo
Una posible firma para la función en python que resuelve el problema es:
’’’N = Número de ascensores
K = Piso a donde me dirijo
T = Tiempo que tarda cada ascensor en viajar entre dos pisos adyacentes
P = Pisos que visita cada ascensor ’’’

def resolver(N,K,T,P):
	pass

Si por ejemplo existen 2 ascensores, y deseo llegar al piso 40, el ascensor #1 visita los pisos 0-1-2-4-6-8-10-12-14-15-21-99 y tarda 10 segundos para viajar entre pisos adyacentes y el ascensor #2 visita los pisos 3-5-10-18-20-25-40 y tarda 5 segundos entre los pisos adyacentes, el sistema calculará 310 segundos ya que tomaría el elevador #1 al piso 10 (100 segundos), esperaría 60 segundos para cambiar al elevador #2 y lo tomo hasta el piso 40 (150 segundos) para un tiempo mínimo total de 310 segundos.


# --------------------------------------------------

## Clase Vértice

Se ha modificado el coste, agregando una casilla más con el número del ascensor en la lista. Además, se han agregado los métodos de get y set.

In [77]:
class Vertex:
    def __init__(self, label):
        self.label = label;
        self.linkedTo = {};
        self.status = False;
        self.coste = [float('inf'), None, -1];
        
    def addAdjacentVertex(self, adjacentVertex, weight, elevator):
        self.linkedTo[adjacentVertex] = [weight, elevator];
        #adjacentVertex.linkedTo[adjacentVertex.getLabel()] = weight
        
    def getConnections(self):
        return self.linkedTo;
    
    def getConnectionValue(self, v_to_label):
        return self.linkedTo.get(v_to_label);
    
    def setConnectionValue(self, v_to_label, value, elevator):
        self.linkedTo[v_to_label] = [value, elevator];
        
    def getLabel(self):
        return self.label;
        
    def getWeight(self, adjacentVertex):
        return self.linkedTo[adjacentVertex];
        
    def getCosteValue(self):
        return self.coste[0];
        
    def getCosteVertex(self):
        return self.coste[1];
        
    def getCosteElevator(self):
        return self.coste[2];
    
    def setCosteElevator(self, elevator):
        self.coste[2] = elevator;
        
    def setCosteInfo(self, value, vertexLabel, elevator):
        self.coste = [value, vertexLabel, elevator];
        
    def setStatus(self, status):
        self.status = status;
        
    def getStatus(self):
        return self.status;

## Clase Grafo

Se ha modificado el método de agregar arista, verificando si existe ya una conexión entre los vértices, si es así, se evalúa cuál es el menor y se deja ese.

In [94]:
class Graph:
    def __init__(self):
        self.vertexSet = {}
        
    def addVertex(self, label):
        newVertex = Vertex(label);
        self.vertexSet[label] = newVertex;
        return newVertex;
        
    def getVertexSet(self):
        return self.vertexSet;
        
    def getVertex(self, label):
        if self.exists(label):
            return self.vertexSet[label];
        return None
        
    def exists(self, label):
        return label in self.vertexSet;
        
    def addEdge(self, vertexFrom, vertexTo, weight = 0, elevator = -1):
        if(self.connectionExists(vertexFrom, vertexTo)):
            currentValue = self.getVertex(vertexFrom).getConnectionValue(vertexTo);
            if(currentValue > weight):
                self.getVertex(vertexFrom).setConnectionValue(vertexTo, weight, elevator);
        else:
            if not self.exists(vertexFrom):
                self.addVertex(vertexFrom);
            if not self.exists(vertexTo):
                self.addVertex(vertexTo);
            self.vertexSet[vertexFrom].addAdjacentVertex(self.vertexSet[vertexTo], weight, elevator);
        
    def getVertexes(self):
        return self.vertexSet.values();
        
    def getVertexLabels(self):
        return iter(self.vertexSet.keys())
    
    def connectionExists(self, v_from, v_to):
        return self.vertexSet[v_from].getConnectionValue(v_to);
    

## Método Dijkstra visto anteriormente
Se ha modificado el método de Dijkstra validando si existe un cambio de ascensor, en ese caso se agregan 60 segundos al costo asociado al movimiento elegido. Sino, se sigue en el mismo elevador y no se agregan segundos al costo total. Además, se ha organizado un poco más el informe de tal manera que se pueda ver el cambio de ascensor.

In [101]:
class Algorithms:
    
    def __init__(self, algorithmId, v_from, v_to, graph):
        self.g = graph;
        if(algorithmId == 1):
            return self.Dijkstra(v_from, v_to, v_from, self.g);
    
    
    #########################################
    ### Dijkstra Algorithm
    #########################################
    def Dijkstra(self, v_from, v_to, originalFrom, G):
        current_v = G.getVertex(v_from);
        # if current_v is the origin vertex, then the coste from itself is 0
        if not current_v.getCosteVertex():
            print("start vertex: ", current_v.getLabel())
            current_v.setCosteInfo(0, current_v.getLabel(), 1);
        print("----------------------------------------------------------------")
        print("----------- processing each connection of ", current_v.getLabel(), " ------------------")
        # for each adjacent vertex from current vertice is necessary to check all values to get the minor and replace
        for v in current_v.getConnections():
            print("Before: ", v.getLabel(), " [", v.getCosteValue(), " - ", v.getCosteVertex(), " - ", v.getCosteElevator(), "]");
            weightInfo = current_v.getWeight(v);
            addChangeElevatorCoste = 0;
            if(current_v.getCosteElevator() != weightInfo[1]):
                addChangeElevatorCoste = 60;
            if (not v.getCosteVertex() or (v.getCosteValue() > (current_v.getCosteValue() + weightInfo[0] + addChangeElevatorCoste))):
                v.setCosteInfo((current_v.getCosteValue() + weightInfo[0] + addChangeElevatorCoste), current_v, weightInfo[1]);
                print("Later: ", v.getLabel(), " [", v.getCosteValue(), " - ", v.getCosteVertex().getLabel(), " - ", weightInfo[1], "]");

        # mark the current_v as visited to discard it for future coste evaluations
        current_v.setStatus(True);
        # Now, it's necessary to get the new curent_v with min value on the graph (no visited vertexes)
        current_v = self.getMinVertexCosteValue();
        
        #if exists the new vertex then process it, but else show a message with no route found
        if(current_v):        
            print("-----------------------------");
            print("new current_v: ", current_v.getLabel())
            
            if(current_v.getLabel() == v_to):
                pathway = self.getPathway(current_v, originalFrom);
                print("----------------------------------------------------------");
                print("----------------------------------------------------------");
                print("La ruta con menor coste desde el piso ", originalFrom, " hasta el piso ", v_to, " es: ", pathway, ", con coste de: ", current_v.getCosteValue());
                print("----------------------------------------------------------");
                print("----------------------------------------------------------");
            else:
                self.Dijkstra(current_v.getLabel(), v_to, originalFrom, G)
        else:
            print("Sin ruta desde ", originalFrom, " hasta ", v_to);
        
    # Method to get the vertex with min value of coste
    def getMinVertexCosteValue(self):
        minValue = float('inf');
        minVertex = None;
        for v in self.g.getVertexes():
            #print("--------------------- Status of ", v.getLabel(), ": ", v.getStatus(), " with value: ", v.getCosteValue());
            if(v.getCosteValue() < minValue and not v.getStatus()):
                minValue = v.getCosteValue();
                minVertex = v;
        return minVertex;
            
    # Method to process the coste value and vertex for pathway
    def getPathway(self, current_v, originalFrom, pathway = ''):
        if(current_v.getLabel() == originalFrom):
            return ("[" + str(current_v.getLabel()) + ", " + str(current_v.getCosteElevator()) + "]" + ' - ' + pathway);
        else:
            if(pathway == ''):
                pathway = "[" + str(current_v.getLabel()) + ", " + str(current_v.getCosteElevator()) + "]";
            else:
                pathway = "[" + str(current_v.getLabel()) + ", " + str(current_v.getCosteElevator()) + "]"  + " - " + pathway;
            return self.getPathway(current_v.getCosteVertex(), originalFrom, pathway);
        

## Configuración del grafo siguiendo las indicaciones de los pisos y los tiempos según el ascensor.

Se agregan los pisos al edificio, los tiempos y los ascensores con sus respectivos pisos.

In [102]:
# building floors are created by loop from 0 to 100 (excluding 100)
buildingFloorsList = [];
for floor in range(0, 100):
    buildingFloorsList.append(floor);

print("---------------------------------------")
print("Building Floors: ", buildingFloorsList);
print("---------------------------------------")

# The time that each elevator takes to go from a floor to another floor is set.
elevatorTimeList = [15, 8, 10];
print("---------------------------------------")
print("Time of elevators: ", elevatorTimeList);
print("---------------------------------------")

# the elevator's array is created with different floor numbers that each elevator can visit
elevatorsList = [
    [0, 2, 4, 7, 11, 13, 16, 20, 25, 27, 31, 36, 38, 45, 49, 51, 53, 59, 62, 66, 68, 70, 71, 74, 78, 83, 87, 89, 92, 93, 95],
    [5, 8, 10, 11, 17, 19, 20, 24, 25, 28, 30, 33, 37, 40, 46, 48, 52, 55, 56, 57, 63, 65, 69, 82, 85, 90, 99],
    [1, 3, 9, 14, 15, 17, 18, 19, 21, 22, 23, 26, 28, 29, 36, 39, 41, 42, 44, 51, 57, 61, 63, 70, 71, 72, 73, 74, 77, 88, 90, 93, 97, 98, 99]
];

print("---------------------------------------")
print("Floor visited by elevators: ");
print("---------------------------------------")
for index, e in enumerate(elevatorsList):
    print("Elevator ", index, "visits: ", e);
    print("")

# Calling function to resolve the problem
print("---------------------------------------")

def Resolve(k, time, floors):
    g = Graph();  
    # First, we need to create all vertex in the graph with building floor list values
    for f in buildingFloorsList:
        g.addVertex(f);

    # Later, for each elevator, we need to add the connections between floors like a graph.
    for elevatorIndex, elevator in enumerate(elevatorsList):
        time = elevatorTimeList[elevatorIndex];
        #for floorIndex, floor in enumerate(elevator[1:]):
        for i in range(1, len(elevator)):
            coste = time * (elevator[i] - elevator[i - 1])
            g.addEdge(elevator[i - 1], elevator[i], coste, (elevatorIndex + 1));
    
    for originLabel, originVertex in g.vertexSet.items():
        for destinationVertex, weight in originVertex.getConnections().items():
            print(str(originLabel) + " - " + str(destinationVertex.getLabel()) + " - " + str(weight))
    pathway = Algorithms(1, 0, k, g);

Resolve(22, elevatorTimeList, buildingFloorsList);


---------------------------------------
Building Floors:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
---------------------------------------
---------------------------------------
Time of elevators:  [15, 8, 10]
---------------------------------------
---------------------------------------
Floor visited by elevators: 
---------------------------------------
Elevator  0 visits:  [0, 2, 4, 7, 11, 13, 16, 20, 25, 27, 31, 36, 38, 45, 49, 51, 53, 59, 62, 66, 68, 70, 71, 74, 78, 83, 87, 89, 92, 93, 95]

Elevator  1 visits:  [5, 8, 10, 11, 17, 19, 20, 24, 25, 28, 30, 33, 37, 40, 46, 48, 52, 55, 56, 57, 63, 65, 69, 82, 85, 90, 99]

Elevator  2 visits